In [ ]:
# #Collector

# import scrapy


# class PlayerSpider(scrapy.Spider):
#     name = 'player'
#     codes=[
#         2,25,1,6,5,7,3,8,4,9,
#         40,19,29,20,30,27,21,104,41,111,108,42,115,
#         12,112,116,117,118,114,43,17,39,44,121,122,
#         4082,123,125,126,13,129,4251,10,131,33,133,
#         35,135,18,137,1094,142,148,147,22,31,36,4083,
#         26,38,159,161,169,16,164,45,165,171,6411,28,32,
#         15,173,175,37,183,178,179,185,187,191,192,154,
#         194,196,23,197,195,200,201,202,4253,204,205,211,
#         207,209,206,34,11,216,215,24,48
#     ]
#     classes=2
#     start_urls=[]
#     for code in codes:
#         start_urls.append('http://www.espncricinfo.com/ci/content/player/caps.html?country={codeem};class={classem}'.format(codeem=code, classem=classes)) 

#     def parse(self, response):
#         for player in response.css('li.sep'):
#             yield {
#                 'text': player.css('a.ColumnistSmry::text').get(),
#                 'link': player.css('a.ColumnistSmry::attr("href")').get(),
#             }




In [ ]:
#extractor

import json
import requests
from bs4 import BeautifulSoup

In [ ]:
readLink = []
with open("player.json", "r") as read_file:
    readLink = json.load(read_file)

start_urls=[]
player_names=[]

for link in readLink:
    dia = link['link']
    dia = dia[12:]
    start_urls.append('http://stats.espncricinfo.com/ci/engine/{val}?class=2;template=results;type=batting'.format(val = dia)) 
    player_names.append(link['text'])

In [ ]:
def check(runs):
    try:
        int(runs)
    except:
        return 'error'

def detcall(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    results = soup.find_all('table', attrs={'class': 'engineTable'})
    result2= results[3].find_all('tbody')
    result3=result2[4].find_all('tr', attrs={'class': 'data1'})
    

    #check year
    yearres=str(result3[1].find('b').text)
    if('year' not in yearres):
        result3=result2[5].find_all('tr', attrs={'class': 'data1'})
    
    result3=result3[1:]

    #check run
    siblingres=[]
    for sibling in result3[0].td.next_siblings:
        siblingres.append(repr(sibling))
#    print(siblingres)
    runyeartagind=9
#    print(runyeartagind)
    runyeartag=siblingres[runyeartagind]
    start=runyeartag.find('<td>')
    end=runyeartag.find('</td>')
    runsres=runyeartag[start+4: end]
    if (check(runsres) == 'error'):
        runyeartagind=7
#        print('changed',runyeartagind)

    dictres={}
    for res in result3:
        #find year
        yearres=str(res.find('b').text)
        # print(yearres)
        # if('year' not in yearres):
        #     return {}
        
        #find runs in that year
        siblingres=[]
        for sibling in res.td.next_siblings:
            siblingres.append(repr(sibling))
        runyeartag=siblingres[runyeartagind]
        start=runyeartag.find('<td>')
        end=runyeartag.find('</td>')
        runsres=runyeartag[start+4: end]
        #add result found to the result dictionary
        if('nowrap' in runsres):
            runsres='0'
        dictres[yearres]=runsres

    return dictres


In [ ]:
final_res={}

# for i in range(len(player_names)):
#     print(player_names[i], start_urls[i], i)
#     detrun=detcall(start_urls[i])
#     print(player_names[i], detrun)
#     final_res[player_names[i]]=detrun

In [ ]:
with open("finalresfile.json", "w") as write_file:
    json.dump(final_res, write_file)